<a href="https://colab.research.google.com/github/MistaAsh/TRINIT_Blank_ML01/blob/main/NLP_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install -q tensorflow_text

     |████████████████████████████████| 4.9 MB 5.1 MB/s 


In [ ]:
pd.set_option('max_colwidth', 200)  # Increase column width
data = pd.read_excel("/content/dataset.xlsx")
data.head()

FileNotFoundError: ignored

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import re

# Preprocessing user inputs with keywords for efficient response mapping
def preprocess_sentences(input_sentences):
    return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I) 
            for input_sentence in input_sentences]
        
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(preprocess_sentences(data.Answer)),
        context=tf.constant(preprocess_sentences(data.Context)))['outputs']

In [ ]:
test_questions = [
    # "What about pregnant women?",
    # "Wat is de lengte van de incubatietijd?",
    # "Are animals contagious COVID-19?",
    # "Are there medicine against the coronavirus?",
    # "Can I breastfead when I have COVID-19?",
    # "Should I stay inside the house?",  # English questions are also possible.
    # "Kann ich mit meinem Hund spazieren gehen?",  # As well as German, and all the other languages supported by use-multilingual.
    # "I am old",
    # " How do I know if it is COVID-19 or just the common flu?",
    # "Can the virus that causes COVID-19 spread to people through the water in saltwater pools?",
    # "Who is at risk of developing severe illness",
    # "should I be worried",
    # "Is there anything I should not do?",
    # "Lmao",
    # "Who is at high risk for serious illness?",
    # "I am pregnant",
    # "old"
    "If I am old what to do?"
]

# Create encodings for test questions
question_encodings = module.signatures['question_encoder'](
    tf.constant(preprocess_sentences(test_questions))
)['outputs']

# Get the responses
test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]

# Show them in a dataframe
pd.DataFrame({'Test Questions': test_questions, 'Test Responses': test_responses})

In [ ]:
def response(input_sentence):
    test_question = [input_sentence]
    question_encodings = module.signatures['question_encoder'](tf.constant(preprocess_sentences(test_questions)))['outputs']

    # Get the response
    test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]
    return test_responses
